In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as pl
pl.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import datacleaning as cl

## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [2]:
verbose = True
    
DATA_PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\" 
# DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/"
    
RAW_PATH = DATA_PATH + "Raw"
PRE_PATH = DATA_PATH + "PreProcessed"     

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [3]:
sessions = []
for user in range(7):
    print ("------- parsing activity files for user ", user, " -------")
    user_sess = paf.get_user_sessions(user, dirname=RAW_PATH)
    print ("------- parsing interval files for user ", user, " -------")
    user_sess = cf.sessions_add_beats(sessions=user_sess, dirname=RAW_PATH+"\\"+str(user), verbose=False)
    sessions.extend(user_sess)

------- parsing activity files for user  0  -------
reading act170929.csv ... 
reading act170930.csv ... 
reading act171001.csv ... 
reading act171003.csv ... 
reading act171004.csv ... 
reading act171005.csv ... 
orphan start in: ['2017-10-05 11:08:11', 'start', 'focused-passive', 'sitting', '']
orphan start in: ['2017-10-05 12:54:00', 'start', 'eat', 'sitting', '']
orphan start in: ['2017-10-05 13:48:39', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 13:57:37', 'start', 'movement', 'standing', '']
orphan start in: ['2017-10-05 17:55:44', 'start', 'focused-active', 'sitting', '']
orphan start in: ['2017-10-05 21:53:45', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 23:47:45', 'start', 'focused-active', 'sitting', 'sitting on bed, installing and using pandas']
reading act171006.csv ... 
reading act171010.csv ... 
orphan start in: ['2017-10-10 23:23:37', 'start', 'leisure-passive', 'sitting']
reading act171011.csv ... 
orphan stop in: ['2017

In [37]:
df = pd.DataFrame(sessions)

### Include column with beats count for ease of use 

In [38]:
df['beatscount'] = df['rr'].apply(len)

In [39]:
df.sample(5)

,activity,duration,notes,posture,rr,start,stop,user,beatscount
248,focused-passive,840,NaN,sit,"[{'date': 2017-10-06 09:23:04, 'interval': 812...",2017-10-06 09:23:00,2017-10-06 09:37:00,2,1094
386,rest-passive,1080,taxi,sit,"[{'date': 2017-12-27 12:52:01, 'interval': 943...",2017-12-27 12:52:00,2017-12-27 13:10:00,5,1174
408,eat,4014,NaN,sit,"[{'date': 2018-01-01 21:09:53, 'interval': 642...",2018-01-01 21:09:52,2018-01-01 22:16:46,6,5483
223,sleep,3120,NaN,lie,"[{'date': 2017-09-30 02:07:29, 'interval': 801...",2017-09-30 02:07:00,2017-09-30 02:59:00,2,1735
224,sleep,900,NaN,lie,"[{'date': 2017-09-30 03:00:01, 'interval': 806...",2017-09-30 03:00:00,2017-09-30 03:15:00,2,925


## 2 - Cleaning data

### 2.1 Removing outliers and artifacts

(Note: For now, beats won't be saved to the output file, so, when loading the sessions file and getting
beats from DB again, for example for fragmentation, they will have to be removed again. But this is a reasonably low-cost op)

In [40]:
df['rr'] = df['rr'].apply(cl.clean_rr_series)

In [41]:
df['beatscount_clean'] = df['rr'].apply(len)

In [42]:
df['removed_artifacts'] = df['beatscount'] - df['beatscount_clean']
df['beatscount'] = df['beatscount_clean']
df = df.drop(['beatscount_clean'], axis=1)

### 2.1 Removing sessions without beats recorded or with too few of them

In [46]:
df = df[df['beatscount'] > 100]

In [47]:
df.describe()

,duration,user,beatscount,removed_artifacts
count,376.000000,376.000000,376.000000,376.000000
mean,2794.885638,1.500000,3471.965426,20.787234
std,5247.313682,1.596663,5966.710217,62.943720
min,88.000000,0.000000,105.000000,0.000000
25%,900.000000,0.000000,1084.750000,0.000000
50%,1299.000000,1.000000,1682.500000,1.000000
75%,2289.750000,2.000000,3201.500000,11.250000
max,34360.000000,6.000000,46856.000000,591.000000


### 2.3 Removing wrong activities and grouping the ones with too few examples

In [48]:
df.at[df['activity']=='exercise-low', 'activity'] = 'exercise'
df.at[df['activity']=='exercise-high', 'activity'] = 'exercise'
df.at[df['activity']=='focused-passive ', 'activity'] = 'focused-passive'
df = df[df['activity']!='not-recorded']

In [49]:
df.groupby('activity').count()['duration']

activity
eat                 43
exercise             6
focused-active      86
focused-passive     37
household-chores    26
leisure-active       6
leisure-passive     45
movement            38
rest-active         24
rest-passive        17
sleep               44
Name: duration, dtype: int64

## 3 - Aggregating data

In [72]:
df['sess_id'] = df.index
df_export = df.drop(['rr'], axis = 1)
df_export.to_excel(PRE_PATH + "\\sessions.xlsx")
df_export.sample(1)

,activity,duration,notes,posture,start,stop,user,beatscount,removed_artifacts,sess_id
198,focused-passive,3491,NaN,sit,2017-11-03 00:00:01,2017-11-03 00:58:12,1,4485,11,198
